## Webscraping the sec.gov

### Goal
Generate a list with all the urls from where to import the Income Statement table.

### Steps

1. download and read tsv files
2. get the 'Directory Listing' from each company (item in tsv)
3. keep only the 'Directory Listing' urls with R (report) files
4. keep only the url for the R (report) that contains the Income Statement


In [2]:
import pandas as pd
import numpy as np
from urllib.request import Request, urlopen
import edgar
import requests
import urllib.request

In [3]:
edgar.download_index(r'/Users/pedrosanhueza/EXOXY/Pedroskis/Programming/SEC_GOV',2018,"Pedro Sanhueza pedroasd1205@gmail.com",skip_all_present_except_last=False)

In [4]:
# read file
file = '2021-QTR1.tsv'
csv = pd.read_csv(file, sep='\t',  lineterminator='\n', names=None)

print("Type: ", type(csv))
print("Rows: ", csv.shape[0])

Type:  <class 'pandas.core.frame.DataFrame'>
Rows:  361139


In [4]:
# Add column name
csv.columns.values[0] = 'Item'

# Keep companies that have 10-k file report
selectedreport = '10-K'
companyreport = csv[(csv['Item'].str.contains(selectedreport))]

# EDA
print("Rows with \"10-K\" forms: ",companyreport.shape[0])
print("Rows dropped: ", csv.shape[0]-companyreport.shape[0])
print("Rows kept from total: ", round(companyreport.shape[0]*100/csv.shape[0],1),"%")

# Showcase result
companyreport.head()

Rows with "10-K" forms:  6032
Rows dropped:  355091
Rows kept from total:  1.7 %


,Item
78,1000209|MEDALLION FINANCIAL CORP|10-K|2021-03-...
96,1000228|HENRY SCHEIN INC|10-K|2021-02-17|edgar...
151,1000229|CORE LABORATORIES N V|10-K|2021-02-08|...
180,1000232|KENTUCKY BANCSHARES INC /KY/|10-K|2021...
772,1000298|IMPAC MORTGAGE HOLDINGS INC|10-K|2021-...


In [5]:
# Convert to object
Filing = companyreport['Item'].str.split('|')
# Convert to list
Filing = Filing.to_list()
Filing[0:3]

[['1000209',
  'MEDALLION FINANCIAL CORP',
  '10-K',
  '2021-03-16',
  'edgar/data/1000209/0001564590-21-013216.txt',
  'edgar/data/1000209/0001564590-21-013216-index.html\r'],
 ['1000228',
  'HENRY SCHEIN INC',
  '10-K',
  '2021-02-17',
  'edgar/data/1000228/0001000228-21-000019.txt',
  'edgar/data/1000228/0001000228-21-000019-index.html\r'],
 ['1000229',
  'CORE LABORATORIES N V',
  '10-K',
  '2021-02-08',
  'edgar/data/1000229/0001564590-21-004561.txt',
  'edgar/data/1000229/0001564590-21-004561-index.html\r']]

In [6]:
# Extract html from list and save them in a different list
count=0
html_list= []
for item in Filing:
    html_list.append('https://www.sec.gov/Archives/' + Filing[count][5])
    html_list[count] = html_list[count].replace("-","").replace('index.html\r','')
    count+=1
print("Looped",count,"times.")

# showcase output
html_list[0:15] 

Looped 6032 times.


['https://www.sec.gov/Archives/edgar/data/1000209/000156459021013216',
 'https://www.sec.gov/Archives/edgar/data/1000228/000100022821000019',
 'https://www.sec.gov/Archives/edgar/data/1000229/000156459021004561',
 'https://www.sec.gov/Archives/edgar/data/1000232/000155837021002326',
 'https://www.sec.gov/Archives/edgar/data/1000298/000155837021002945',
 'https://www.sec.gov/Archives/edgar/data/1000623/000100062321000047',
 'https://www.sec.gov/Archives/edgar/data/1000683/000121390021017745',
 'https://www.sec.gov/Archives/edgar/data/1000694/000100069421000004',
 'https://www.sec.gov/Archives/edgar/data/1000697/000119312521054385',
 'https://www.sec.gov/Archives/edgar/data/1000753/000100075321000009',
 'https://www.sec.gov/Archives/edgar/data/1001082/000155837021001322',
 'https://www.sec.gov/Archives/edgar/data/1001171/000114036121011100',
 'https://www.sec.gov/Archives/edgar/data/100122/000010012221000005',
 'https://www.sec.gov/Archives/edgar/data/1001233/000162828021003069',
 'https

In [7]:
# %%timeit 5.01 s ± 985 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# Keep only those url with R's files (ex. R1.htm)
user_agent = 'jordan.tway@hcminst.com'
headers={'User-Agent':user_agent,
         'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
         'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
         'Accept-Encoding': 'none',
         'Accept-Language': 'en-US,en;q=0.8',
         'Connection': 'keep-alive'} 
elements_in_html_list=0
count=0
contains=0
not_contains=0
html_list_FillingSummary = []


# loop over each url and keep those that have the "R.htm" tables
# only using first 20 companies for testing purposes
for e in range(0, len(html_list)):
    elements_in_html_list+=1
    url = html_list[count]
    request=urllib.request.Request(url,None,headers)
    response = urllib.request.urlopen(request)
    data = response.read()
    df = pd.read_html(data)
    if df[0]['Name'].str.contains("R1.htm").any():
        html_list_FillingSummary.append(html_list[contains])
#         print(contains,"added", url)
        contains+=1
    else:
#         print("index: ", contains, "not working")
        not_contains+=1
        contains+=1
    count+=1

print("\n")
print(count,"Companies read")
print(not_contains,"Companies w/out 10-K")
print(len(html_list_FillingSummary), "URL's kept")



6032 Companies read
1418 Companies w/out 10-K
4614 URL's kept


In [8]:
# %%timeit 377 µs ± 21.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
# Add R file extention to the URL's (-new list-)

elements_in_html_list=0
count=0
total=0
html_R_list = []

for e in range(0, len(html_list_FillingSummary)):
    elements_in_html_list+=1
    r_count=1
    for i in range(0,5):
        url = html_list_FillingSummary[count] + "/R" + str(r_count) + ".htm"
        html_R_list.append(url)
        r_count+=1
        total+=1
    count+=1

print("Loops in \"html_list_FillingSummary\": ",elements_in_html_list)
print("Loops overall",total)
print("Companies read: ",count)
print("URL list length:", len(html_R_list))

# Companies with their first 4 R files
html_R_list[0:10]

Loops in "html_list_FillingSummary":  4614
Loops overall 23070
Companies read:  4614
URL list length: 23070


['https://www.sec.gov/Archives/edgar/data/1000209/000156459021013216/R1.htm',
 'https://www.sec.gov/Archives/edgar/data/1000209/000156459021013216/R2.htm',
 'https://www.sec.gov/Archives/edgar/data/1000209/000156459021013216/R3.htm',
 'https://www.sec.gov/Archives/edgar/data/1000209/000156459021013216/R4.htm',
 'https://www.sec.gov/Archives/edgar/data/1000209/000156459021013216/R5.htm',
 'https://www.sec.gov/Archives/edgar/data/1000228/000100022821000019/R1.htm',
 'https://www.sec.gov/Archives/edgar/data/1000228/000100022821000019/R2.htm',
 'https://www.sec.gov/Archives/edgar/data/1000228/000100022821000019/R3.htm',
 'https://www.sec.gov/Archives/edgar/data/1000228/000100022821000019/R4.htm',
 'https://www.sec.gov/Archives/edgar/data/1000228/000100022821000019/R5.htm']

In [9]:
len(html_R_list)

23070

In [33]:
# %%timeit 22.2 s ± 1.03 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
# Open each url and look for income statement
# append the correct urls to a new list

elements_in_html_list=0
contains=0
count=0
total=0
loops = 0
index=0
nothing=0
list_R_URL_filtered = []

for e in html_R_list:
# read url
    url = html_R_list[count]
    request=urllib.request.Request(url,None,headers)
    response = urllib.request.urlopen(request)
    data = response.read()
    df = pd.read_html(data)
# check for Income Statement
    for e in df[0][0:1]:
        loops+=1
        if "COMPREHENSIVE" in e[index]:
#             print("Comprehensive in index:", count)
            break
        elif "Comprehensive" in e[index]:
#             print("Comprehensive in index:", count)
            break
        elif "Parenthetical" in e[index]:
#             print("Parenthetical in index:", count)
            break
        elif "Operations" in e[index]:
            print("Index: ",count, ", Operations Appended--:", url)
            list_R_URL_filtered.append(url)
            break
        elif "Income" in e[index]:
            print("Index: ",count, ", Income Appended------:", url)
            list_R_URL_filtered.append(url)
            break
        elif "OPERATIONS" in e[index]:
            print("Index: ",count, ", Operations Appended--:", url)
            list_R_URL_filtered.append(url)
            break
        elif "INCOME" in e[index]:
            print("Index: ",count, ", Income Appended------:", url)
            list_R_URL_filtered.append(url)
            break
        else:
#             print("Nothing in index", count)
            nothing+=1
            break
    count+=1
print(loops, "Loops")

Index:  3 , Operations Appended--: https://www.sec.gov/Archives/edgar/data/1000209/000156459021013216/R4.htm
Index:  8 , Income Appended------: https://www.sec.gov/Archives/edgar/data/1000228/000100022821000019/R4.htm
Index:  13 , Operations Appended--: https://www.sec.gov/Archives/edgar/data/1000229/000156459021004561/R4.htm
Index:  18 , Income Appended------: https://www.sec.gov/Archives/edgar/data/1000232/000155837021002326/R4.htm
19 Loops


In [11]:
len(list_R_URL_filtered)

15

In [12]:
# see end result
print(len(list_R_URL_filtered), "urls kept")
print("First 10 items in list:")
for i in list_R_URL_filtered[0:10]:
    print(i)

15 urls kept
First 10 items in list:
https://www.sec.gov/Archives/edgar/data/1000209/000156459021013216/R4.htm
https://www.sec.gov/Archives/edgar/data/1000228/000100022821000019/R4.htm
https://www.sec.gov/Archives/edgar/data/1000229/000156459021004561/R4.htm
https://www.sec.gov/Archives/edgar/data/1000232/000155837021002326/R4.htm
https://www.sec.gov/Archives/edgar/data/1000623/000100062321000047/R2.htm
https://www.sec.gov/Archives/edgar/data/1000694/000100069421000004/R4.htm
https://www.sec.gov/Archives/edgar/data/1000697/000119312521054385/R4.htm
https://www.sec.gov/Archives/edgar/data/1000753/000100075321000009/R3.htm
https://www.sec.gov/Archives/edgar/data/100122/000010012221000005/R2.htm
https://www.sec.gov/Archives/edgar/data/1001233/000162828021003069/R4.htm


In [13]:
# Save with np

np.savetxt("urls2.csv", 
           list_R_URL_filtered,
           delimiter =", ", 
           fmt ='% s')

In [14]:
# url = list_R_URL_filtered[8]
# request=urllib.request.Request(url,None,headers)
# response = urllib.request.urlopen(request)
# data = response.read()
# df = pd.read_html(data)[0] # first data frame of the list

In [15]:
# df.head()

In [16]:
# print(url)
# df.columns[3]

In [17]:
# type(df.columns[0])

In [18]:
# df.columns[0]

In [19]:
# heading_list = list (df.columns[0])
# heading_list

In [20]:
# type(heading_list)

In [21]:
# del heading_list[0]
# heading_list

In [22]:
# heading_list = tuple(heading_list)

In [23]:
# type(heading_list)

In [24]:
# df.columns.values[0] = "Yes"

In [25]:
# df['CIK'] = url.split('/')[7]

In [26]:
# df1= df
# df1.head()

In [27]:
# df2 = pd.concat([df, df1])

In [28]:
# df2

In [29]:
# df.head()